In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from tqdm import tqdm

from shamans.utils import eval_utils as eval

In [18]:
# merge all the csv fo the experiments 1 for different runs
def merge_csvs_and_get_dataframe(exp_id, path_to_results):
    all_filenames = [i for i in path_to_results.glob(f'*_exp-{exp_id}_run-*.csv')]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
    combined_csv.to_csv(path_to_results / f"experiment_results_exp-{exp_id}_all_runs.csv", index=False, encoding='utf-8-sig')
    return combined_csv

In [19]:
# load data
exp_id = 1
path_to_results = Path('results/')
exp_df = merge_csvs_and_get_dataframe(exp_id, path_to_results)

In [20]:
# compute accuracy and recall for each doas estimated by each method within a threshold of 10 degrees

def calc_ae(a,b):
    x = np.abs(a-b)
    return np.min(np.array((x, np.abs(360-x))), axis=0)

exp_df['ang_error'] = calc_ae(
    np.rad2deg(exp_df['doas_est_az']), 
    np.rad2deg(exp_df['doas_ref_az'])
)

,exp_name,time,record_id,num_srcs,src_ids,doas_est_idx,doas_ref_idx,doas_ref_az,doas_est_az,doas_ref_el,...,mc_seed,method_id,loc_method,freq_min,freq_max,sv_method,nObs,seed,sv_normalization,ang_error
0,exp-1_nSrc-1_doas-[44]_type-speech-duration-0....,20250307-155857,s0,1,0,4,44,4.607669,0.418879,1.570796,...,3,"alpha-1.2_beta-2_eps-1E-3_iter-500_freqs-[200,...",alpha-1.2_beta-2_eps-1E-3_iter-500,200,4000,ref,8,13,True,120.0
1,exp-1_nSrc-1_doas-[44]_type-speech-duration-0....,20250307-155857,s0,1,0,19,44,4.607669,1.989675,1.570796,...,3,"alpha-1.2_beta-2_eps-1E-3_iter-500_freqs-[200,...",alpha-1.2_beta-2_eps-1E-3_iter-500,200,4000,alg,8,13,True,150.0
2,exp-1_nSrc-1_doas-[44]_type-speech-duration-0....,20250307-155857,s0,1,0,4,44,4.607669,0.418879,1.570796,...,3,"alpha-1.2_beta-2_eps-1E-3_iter-500_freqs-[200,...",alpha-1.2_beta-2_eps-1E-3_iter-500,200,4000,gp-steerer,8,13,True,120.0
3,exp-1_nSrc-1_doas-[44]_type-speech-duration-0....,20250307-155857,s0,1,0,44,44,4.607669,4.607669,1.570796,...,3,"alpha-1.2_beta-2_eps-1E-3_iter-500_freqs-[200,...",alpha-1.2_beta-2_eps-1E-3_iter-500,200,4000,gp-steerer,8,42,True,0.0
4,exp-1_nSrc-1_doas-[44]_type-speech-duration-0....,20250307-155857,s0,1,0,6,44,4.607669,0.628319,1.570796,...,3,"alpha-1.2_beta-2_eps-1E-3_iter-500_freqs-[200,...",alpha-1.2_beta-2_eps-1E-3_iter-500,200,4000,gp-steerer,8,666,True,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,exp-1_nSrc-1_doas-[5]_type-speech-duration-0.5...,20250307-160726,s0,1,0,5,5,0.523599,0.523599,1.570796,...,4,"srp_phat_freqs-[200, 4000]_gp-steerer_nObs-64_...",srp_phat,200,4000,gp-steerer,64,42,True,0.0
1067,exp-1_nSrc-1_doas-[5]_type-speech-duration-0.5...,20250307-160726,s0,1,0,5,5,0.523599,0.523599,1.570796,...,4,"srp_phat_freqs-[200, 4000]_gp-steerer_nObs-64_...",srp_phat,200,4000,gp-steerer,64,666,True,0.0
1068,exp-1_nSrc-1_doas-[5]_type-speech-duration-0.5...,20250307-160726,s0,1,0,5,5,0.523599,0.523599,1.570796,...,4,"srp_phat_freqs-[200, 4000]_gp-steerer_nObs-128...",srp_phat,200,4000,gp-steerer,128,13,True,0.0
1069,exp-1_nSrc-1_doas-[5]_type-speech-duration-0.5...,20250307-160726,s0,1,0,5,5,0.523599,0.523599,1.570796,...,4,"srp_phat_freqs-[200, 4000]_gp-steerer_nObs-128...",srp_phat,200,4000,gp-steerer,128,42,True,0.0
